# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [1]:
# Write your code below.

%load_ext dotenv
%dotenv 

In [2]:
import dask.dataframe as dd

/var/folders/qt/l2qz_x2s25s9bn9zhf9ynxp40000gn/T/ipykernel_43770/676544213.py:1: DeprecationWarning: The current Dask DataFrame implementation is deprecated. 
In a future release, Dask DataFrame will use new implementation that
contains several improvements including a logical query planning.
The user-facing DataFrame API will remain unchanged.

The new implementation is already available and can be enabled by
installing the dask-expr library:

    $ pip install dask-expr

and turning the query planning option on:

    >>> import dask
    >>> dask.config.set({'dataframe.query-planning': True})
    >>> import dask.dataframe as dd

API documentation for the new implementation is available at
https://docs.dask.org/en/stable/dask-expr-api.html

Any feedback can be reported on the Dask issue tracker
https://github.com/dask/dask/issues 

  import dask.dataframe as dd


+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [3]:
import os
from glob import glob

# Write your code below.
price_data = os.getenv("PRICE_DATA")
# Find all Parquet files in the PRICE_DATA directory
parquet_files = glob(os.path.join(price_data, "*/*/*.parquet"))
parquet_files

['../../05_src/data/prices/CTAS/CTAS_2008.parquet/part.0.parquet',
 '../../05_src/data/prices/CTAS/CTAS_2018.parquet/part.0.parquet',
 '../../05_src/data/prices/CTAS/CTAS_2011.parquet/part.0.parquet',
 '../../05_src/data/prices/CTAS/CTAS_2001.parquet/part.0.parquet',
 '../../05_src/data/prices/CTAS/CTAS_2000.parquet/part.0.parquet',
 '../../05_src/data/prices/CTAS/CTAS_2010.parquet/part.0.parquet',
 '../../05_src/data/prices/CTAS/CTAS_2019.parquet/part.0.parquet',
 '../../05_src/data/prices/CTAS/CTAS_2009.parquet/part.0.parquet',
 '../../05_src/data/prices/CTAS/CTAS_2012.parquet/part.0.parquet',
 '../../05_src/data/prices/CTAS/CTAS_2002.parquet/part.0.parquet',
 '../../05_src/data/prices/CTAS/CTAS_2024.parquet/part.0.parquet',
 '../../05_src/data/prices/CTAS/CTAS_2003.parquet/part.0.parquet',
 '../../05_src/data/prices/CTAS/CTAS_2013.parquet/part.0.parquet',
 '../../05_src/data/prices/CTAS/CTAS_2016.parquet/part.0.parquet',
 '../../05_src/data/prices/CTAS/CTAS_2006.parquet/part.0.parqu

For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Adjusted Close:
    
    - `returns`: (Adj Close / Adj Close_lag) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [4]:
# Write your code below.
import numpy as np
dd_px = dd.read_parquet(parquet_files).reset_index()
dd_px.head()

,ticker,Date,Open,High,Low,Close,Adj Close,Volume,sector,subsector,year
0,A,2000-01-03,56.330471,56.464592,48.193848,51.502148,43.463020,4674353,Health Care,Life Sciences Tools & Services,2000
1,A,2000-01-04,48.730328,49.266811,46.316166,47.567955,40.142937,4765083,Health Care,Life Sciences Tools & Services,2000
2,A,2000-01-05,47.389126,47.567955,43.141991,44.617310,37.652874,5758642,Health Care,Life Sciences Tools & Services,2000
3,A,2000-01-06,44.080830,44.349072,41.577251,42.918453,36.219193,2534434,Health Care,Life Sciences Tools & Services,2000
4,A,2000-01-07,42.247852,47.165592,42.203148,46.494991,39.237465,2819626,Health Care,Life Sciences Tools & Services,2000


In [5]:
import numpy as np
dd_feat = (dd_px.groupby('ticker', group_keys=False).apply(
    lambda x: x.assign(Close_lag_1 = x['Close'].shift(1))
).assign(
    returns = lambda x: x['Close']/x['Close_lag_1'] - 1
).assign(
    positive_return = lambda x: (x['returns'] > 0)*1
).assign(
    hi_lo_range = lambda x: (x['High'] -x['Low'])
))

/var/folders/qt/l2qz_x2s25s9bn9zhf9ynxp40000gn/T/ipykernel_43770/734013228.py:2: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_feat = (dd_px.groupby('ticker', group_keys=False).apply(


In [6]:
dd_feat.head()

,ticker,Date,Open,High,Low,Close,Adj Close,Volume,sector,subsector,year,Close_lag_1,returns,positive_return,hi_lo_range
0,HUM,2004-01-02,23.049999,23.240000,22.860001,22.930000,20.474092,837700,Health Care,Managed Health Care,2004,NaN,NaN,0,0.379999
1,HUM,2004-01-05,21.299999,22.400000,20.860001,21.389999,19.099037,3070300,Health Care,Managed Health Care,2004,22.930000,-0.067161,0,1.539999
2,HUM,2004-01-06,21.299999,21.840000,20.910000,21.620001,19.304401,1462100,Health Care,Managed Health Care,2004,21.389999,0.010753,1,0.930000
3,HUM,2004-01-07,21.700001,21.799999,21.200001,21.420000,19.125826,1149100,Health Care,Managed Health Care,2004,21.620001,-0.009251,0,0.599998
4,HUM,2004-01-08,21.270000,21.290001,20.570000,20.889999,18.652588,1998100,Health Care,Managed Health Care,2004,21.420000,-0.024743,0,0.720001


+ Convert the Dask data frame to a pandas data frame. 
+ Add a rolling average return calculation with a window of 10 days.
+ *Tip*: Consider using `.rolling(10).mean()`.

(3 pt)

In [8]:
pd_feat =dd_feat.compute()

In [9]:
pd_feat['rolling_avg_return'] = pd_feat['returns'].rolling(window=10).mean()
pd_feat

,ticker,Date,Open,High,Low,Close,Adj Close,Volume,sector,subsector,year,Close_lag_1,returns,positive_return,hi_lo_range,rolling_avg_return
0,HUM,2004-01-02,23.049999,23.240000,22.860001,22.930000,20.474092,837700,Health Care,Managed Health Care,2004,NaN,NaN,0,0.379999,NaN
1,HUM,2004-01-05,21.299999,22.400000,20.860001,21.389999,19.099037,3070300,Health Care,Managed Health Care,2004,22.930000,-0.067161,0,1.539999,NaN
2,HUM,2004-01-06,21.299999,21.840000,20.910000,21.620001,19.304401,1462100,Health Care,Managed Health Care,2004,21.389999,0.010753,1,0.930000,NaN
3,HUM,2004-01-07,21.700001,21.799999,21.200001,21.420000,19.125826,1149100,Health Care,Managed Health Care,2004,21.620001,-0.009251,0,0.599998,NaN
4,HUM,2004-01-08,21.270000,21.290001,20.570000,20.889999,18.652588,1998100,Health Care,Managed Health Care,2004,21.420000,-0.024743,0,0.720001,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247,JNPR,2004-12-27,27.040001,27.160000,26.680000,26.850000,20.923876,3032500,Information Technology,Communications Equipment,2004,26.879999,-0.001116,0,0.480000,-0.004586
248,JNPR,2004-12-28,26.900000,27.049999,26.709999,27.030001,21.064146,3401700,Information Technology,Communications Equipment,2004,26.850000,0.006704,1,0.340000,-0.003454
249,JNPR,2004-12-29,27.000000,27.350000,27.000000,27.100000,21.118698,3135900,Information Technology,Communications Equipment,2004,27.030001,0.002590,1,0.350000,-0.000731
250,JNPR,2004-12-30,27.049999,27.250000,26.940001,27.170000,21.173241,2856500,Information Technology,Communications Equipment,2004,27.100000,0.002583,1,0.309999,-0.001498


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?


No,  Dask is designed to handle large datasets that do not fit in memory. When I converted it to pandas, it took me forever to run the compute().
+ Would it have been better to do it in Dask? Why?


Yes, as my answer before, dask is designed to handle large datasets that do not fit in memory. and Pandas is better for smaller datasets.

(1 pt)

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.